# Check if all csv files are present
  
For all patient folders, check if all csv files are present.  

## 1. Loading extensions and necessary libraries

These cells load the `autoreload` extension to automatically reload all modules before executing the Python code, and imports the necessary libraries for data processing and graphical visualization.

In [1]:
# Load the autoreload extension
%load_ext autoreload

In [ ]:
# Set IPython magic extension to autoreload all modules every time before executing the Python code
%autoreload 2

# Standard library imports
import sys
import os
import re

# Install required libraries
# If some of these libraries are already installed and you don't want to install 
# them again, please comment the concerned lines in this block

# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install IPython

# Third party imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.signal import butter, filtfilt

# matplotlib config
mpl.rcParams['agg.path.chunksize'] = 10000

## 2. Configure working directory and source paths

This block ensures that the working directory is set to the project root and that the `sources` folder is added to the Python path to allow proper imports. It also defines the path to the actimetry data folder.

In [3]:
# Configure working directory and source paths
notebook_dir = os.getcwd()
sys.path.append(os.path.join(notebook_dir,"..", "sources"))

# If the present working directory is the directory of this file, change the path to the project root
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Path to actimetry data
act_folder = "data/data_actimetry/"

## 3. View patients metadata
`participants_X.csv` :   
- `X` is the ID of the month of record (from `1` = first month to `6` = sixth month)
- The file contains, for each patient: 
    - `folder_name` : Patient ID (e.g., C1P30 corresponding to investigation center 1 and the 30th patient included in the study)
    - `is_patient` : Patient (True) or healthy subject (False)
    - `parent_folder` : Actimetric data folder name
    - `paretic_side` : Right or left paretic side
    - `start_day`, `start_month`, `start_year` : Start date of recording
    - `end_day`, `end_month`, `end_year` : End date of recording
    - `age` : Subject's age on the day of inclusion
    - `FMScore` : The Fugl-Meyer score (upper-limb function, /66)
    - `freq` : Sampling frequency
    - `time_stroke` : Number of months post-stroke
    - `laterality` : Laterality
    - `barthel` : Barthel score (autonomy index, /100)
    - `bbt_paretic` and `bbt_non_paretic` : Box and Block test scores (gross motor function)

In [4]:
# Define the metadata file path
participants_csv_path = "data/participants_1.csv"

# Check if the CSV file exists
if not os.path.exists(participants_csv_path):
    raise FileNotFoundError(f"The file {participants_csv_path} was not found.")

# Load participants metadata
participants_info = pd.read_csv(participants_csv_path, sep=';')

# Display sample of metadata
print("\nTable 1. Patients metadata")
display(participants_info.head())


Table 1. Patients metadata


,folder_name,is_patient,parent_folder,paretic_side,start_day,end_day,start_month,end_month,start_year,end_year,age,FMScore,freq,time_stroke,laterality,barthel,bbt_paretic,bbt_non_paretic
0,C1P01_M1,True,data_actimetry,left,21,29,3,3,2025,2025,23,33,50,121,right,100,0,64
1,C1P30_M1,True,data_actimetry,right,29,7,11,12,2024,2024,67,45,50,36,right,95,17,46
2,C1P31_M1,True,data_actimetry,left,10,18,1,1,2025,2025,75,49,50,123,right,80,35,52
3,C1P32_M1,True,data_actimetry,right,29,7,11,12,2024,2024,78,45,50,121,right,90,20,50
4,C1P33_M1,True,data_actimetry,left,29,7,11,12,2024,2024,81,54,50,33,right,85,18,38


## 4. Define months you want to process
`months` is a list of integers representing the months to analyze, from 1 (first month) to 6 (sixth month).

In [5]:
# Months to analyse
months = [1]

## 5. Check if all csv files are present
For all patient folders, check if all csv files are present.  

In [6]:
# Definition of the base path and the list of patients
patients = participants_info['folder_name'].str.split('_M').str[0].tolist()
months_m = [f"M{m}" for m in months]

print("\nVERIFICATION OF THE PRESENCE OF ALL ACTIMETRIC DATA FILES:")
print(f"Checked patients: {', '.join(patients)}")
print(f"Checked months: {', '.join(map(str, months))}")

def check_files():
    """
    Check if the files are present in the base path for each patient and each month.
    """
    all_files_present = True

    # Loop through each patient and each month
    for patient in patients:
        for month in months_m:
            folder_name = f"{patient}_{month}"
            folder_path = os.path.join(act_folder, folder_name)
            
            # Check if the folder exists
            if not os.path.exists(folder_path):
                print(f"***** Missing folder: {folder_name} *****")
                all_files_present = False
                continue
            
            # Define paths to left and right CSV files            
            left_file = os.path.join(folder_path, "left.csv")
            right_file = os.path.join(folder_path, "right.csv")

            # Check if each file exists
            missing_files = []
            if not os.path.isfile(left_file):
                missing_files.append("left.csv")
            if not os.path.isfile(right_file):
                missing_files.append("right.csv")

            # Print missing files
            if missing_files:
                print(f"***** Missing files in {folder_name}: {', '.join(missing_files)} *****")
                all_files_present = False

    if all_files_present:
        print("All files are present.")

check_files()

print("\n")


VERIFICATION OF THE PRESENCE OF ALL ACTIMETRIC DATA FILES:
Checked patients: C1P01, C1P30, C1P31, C1P32, C1P33
Checked months: 1
***** Missing folder: C1P30_M1 *****
***** Missing folder: C1P31_M1 *****
***** Missing folder: C1P32_M1 *****
***** Missing folder: C1P33_M1 *****


